In [3]:
import pandas as pd
from rapidfuzz import fuzz
import re

In [4]:
# Changed to index_col=[0] to prevent Unnamed: X columns
df = pd.read_csv("company_website_second_round_with_additional_firms.csv", low_memory=True, index_col=[0])
df = df.drop(columns=[col for col in df.columns if col.startswith('Unnamed:')]) # If th

non_temporal_df = df.iloc[:, 0:13] # Will need to change indexing if more non-temporal columns are added
df = df.drop(df.columns[0:13], axis=1)
assert not any(re.match(r"\d{4}-\d{2}", col) for col in non_temporal_df.columns), "Columns with the pattern 'YYYY-MM' are present in the non-temporal dataframe"
assert all(re.match(r"\d{4}-\d{2}", col) for col in df.columns), "Not all columns follow the 'YYYY-MM' pattern"

/tmp/ipykernel_144/3174632621.py:2: DtypeWarning: Columns (300,304,325,328) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("company_website_second_round_with_additional_firms.csv", low_memory=True, index_col=[0])


In [5]:
df.shape

(5189, 325)

In [6]:
def compute_hash(string):
    ans = 0
    for i in range(0, min(len(string), 8)):
        ans += ord(string[i]) * (7**i)
    return ans

In [7]:
table = {}
saved = 0
considering = 0
for index, row in enumerate(df.itertuples()):
    if index % 100 == 0:
        print(f"Finished processing {index} out of {len(df.index)} rows.")
    # print(len(row))
    for column_index, value in enumerate(row):
        if pd.isna(value) or type(value) == int or type(value) == float:
            continue
        value_hash = compute_hash(value)
        
        if not(value_hash in table):
            table[value_hash] = []

        max_similarity = 0
        for existing_value in table[value_hash]:
            max_similarity = max(max_similarity, fuzz.ratio(existing_value, value))

        if max_similarity < 0.95:
            table[value_hash].append(value)
            considering += 1
        else:
            saved += 1
            # print(index, column_index)
            df.iloc[index, column_index - 1] = -1 # subtracting 1 from the column_index because the first one is probably something irrelevant
            
print(saved, considering)

Finished processing 0 out of 5189 rows.
Finished processing 100 out of 5189 rows.
Finished processing 200 out of 5189 rows.
Finished processing 300 out of 5189 rows.
Finished processing 400 out of 5189 rows.
Finished processing 500 out of 5189 rows.
Finished processing 600 out of 5189 rows.
Finished processing 1500 out of 5189 rows.
Finished processing 1600 out of 5189 rows.
Finished processing 1700 out of 5189 rows.
Finished processing 1800 out of 5189 rows.
Finished processing 1900 out of 5189 rows.
Finished processing 2000 out of 5189 rows.
Finished processing 2100 out of 5189 rows.
Finished processing 2200 out of 5189 rows.
Finished processing 2300 out of 5189 rows.
Finished processing 2400 out of 5189 rows.
Finished processing 2500 out of 5189 rows.
Finished processing 2600 out of 5189 rows.
Finished processing 2700 out of 5189 rows.
Finished processing 2800 out of 5189 rows.
Finished processing 2900 out of 5189 rows.
Finished processing 3000 out of 5189 rows.
Finished processing 

In [8]:
actual_text_count = 0

for index, row in enumerate(df.itertuples()):
    if index % 100 == 0:
        print(f"Finished processing {index} out of {len(df.index)} rows.")
    # print(len(row))
    for column_index, value in enumerate(row):
        if pd.isna(value) or type(value) == int or type(value) == float:
            continue
        actual_text_count += 1

print(actual_text_count)

Finished processing 0 out of 5189 rows.
Finished processing 100 out of 5189 rows.
Finished processing 200 out of 5189 rows.
Finished processing 300 out of 5189 rows.
Finished processing 400 out of 5189 rows.
Finished processing 500 out of 5189 rows.
Finished processing 600 out of 5189 rows.
Finished processing 700 out of 5189 rows.
Finished processing 800 out of 5189 rows.
Finished processing 900 out of 5189 rows.
Finished processing 1000 out of 5189 rows.
Finished processing 1100 out of 5189 rows.
Finished processing 1200 out of 5189 rows.
Finished processing 1300 out of 5189 rows.
Finished processing 1400 out of 5189 rows.
Finished processing 1500 out of 5189 rows.
Finished processing 1600 out of 5189 rows.
Finished processing 1700 out of 5189 rows.
Finished processing 1800 out of 5189 rows.
Finished processing 1900 out of 5189 rows.
Finished processing 2000 out of 5189 rows.
Finished processing 2100 out of 5189 rows.
Finished processing 2200 out of 5189 rows.
Finished processing 230

In [9]:
def get_phrases(corpus, length=20, split=15):
    total_words = [word.lower() for word in corpus.split()]
    phrases = []
    
    for i in range(0, len(total_words), split):
        right_idx = min(len(total_words), i + length)
        phrases.append(" ".join(total_words[i:right_idx]))
    
    return phrases

In [10]:
from sentence_transformers import SentenceTransformer, util
embeddings_model = SentenceTransformer("all-MiniLM-L6-v2")

/opt/conda/lib/python3.11/site-packages/torch/_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()


In [63]:
import json

existing_phrases_file = open("antiforeign_revised_2_phrases.json", "r")
existing_phrases = json.loads(existing_phrases_file.read())[:10]
existing_embedded = embeddings_model.encode(existing_phrases, convert_to_tensor=True)

In [64]:
import torch
import numpy as np
import time

counter = 0
time_last = time.time()
similar_phrases = []

non_llm_map = {}
keyword = "antiforeign"
total_phrase_count = 0

def count_phrases(text):
    global time_last
    global counter 
    global similar_phrases
    global base_embedded
    global total_phrase_count
    
    if pd.isna(text) or type(text) == float or type(text) == int:
        return text
        
    phrases = get_phrases(text)
    if len(phrases) == 0:
        phrases = [text]

    phrase_count = 0
    phrases_embedded = embeddings_model.encode(phrases, convert_to_tensor=True)
    
    # Compute cosine-similarities for each sentence with each other sentence
    cosine_scores = util.cos_sim(existing_embedded, phrases_embedded)
    
    # Find the pairs with the highest cosine similarity scores
    for i in range(cosine_scores.shape[0]):
        for j in range(cosine_scores.shape[1]):
            if cosine_scores[i][j] > 0.4:
                phrase_count += 1
                total_phrase_count += 1
    
    counter += 1

    if counter % 500 == 0:
        print(f"Processed {counter} cells in {time.time() - time_last}")
        time_last = time.time()
    
    return phrase_count

In [65]:
# import json
# test_text = open("test.txt", "r").read()
# for keyword_file in ["antiforeign_revised_2_phrases.json", "jobs_revised_phrases.json", "labor_revised_phrases.json", "military_revised_phrases.json", "national_pride_revised_phrases.json", "quality_revised_phrases.json"]:
#     contents = open(keyword_file).read()
#     contents = json.loads(contents)

    # keyword = keyword_file.split("_")[0]
    # base_embedded = embeddings_model.encode(contents[:10], convert_to_tensor=True)
    # print(keyword, count_phrases(test_text))

In [66]:
all_similar_phrases = {}

In [67]:
print("Current keyword being processed: ", keyword)
df2 = df.map(count_phrases)
all_similar_phrases[keyword] = similar_phrases

Current keyword being processed:  antiforeign
Processed 500 cells in 13.8411123752594
Processed 1000 cells in 15.03171706199646
Processed 1500 cells in 13.863390684127808
Processed 2000 cells in 13.12397313117981
Processed 2500 cells in 14.801108598709106
Processed 3000 cells in 12.41403865814209
Processed 3500 cells in 12.560847043991089
Processed 4000 cells in 12.678217887878418
Processed 4500 cells in 15.222747087478638
Processed 5000 cells in 12.477580547332764
Processed 5500 cells in 11.695727825164795
Processed 6000 cells in 11.638166427612305
Processed 6500 cells in 10.10848069190979
Processed 7000 cells in 9.641452550888062
Processed 7500 cells in 8.280066967010498
Processed 8000 cells in 9.580994606018066
Processed 8500 cells in 9.09194016456604
Processed 9000 cells in 8.186546325683594
Processed 9500 cells in 7.073901176452637
Processed 10000 cells in 7.544878005981445
Processed 10500 cells in 7.716348648071289
Processed 11000 cells in 6.433673620223999
Processed 11500 cells 

In [68]:
df2.head()

,2023-10,2023-09,2023-08,2023-07,2023-06,2023-05,2023-04,2023-03,2023-02,2023-01,...,1997-07,1997-06,1997-05,1997-04,1997-03,1997-02,1997-01,1996-12,1996-11,1996-10
Unnamed: 0,,,,,,,,,,,,,,,,,,,,,
0,NaN,NaN,NaN,NaN,NaN,NaN,10.0,-1.0,-1.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,1.0,-1.0,NaN,-1.0,NaN,-1.0,-1.0,-1.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,6.0,NaN,NaN,-1.0,NaN,-1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,6.0,-1.0,4.0,NaN,NaN,-1.0,NaN,-1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,-1.0,-1.0,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,36.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [69]:
df2.to_csv(f"count_csvs/{keyword}_simsearch_nofix.csv")

In [70]:
total_phrase_count

32162

In [71]:
df2 = pd.read_csv(f"count_csvs/{keyword}_simsearch_nofix.csv")

In [72]:
df2 = df2.drop(columns=[col for col in df2.columns if col.startswith('Unnamed:')]) # If th

In [73]:
for index, row in df2.iterrows():
    prev_value = None
    for col in reversed(df2.columns):
        value = row[col]
        if pd.isna(value) or value == -1 or value == 0:
            if prev_value is not None:
                df2.at[index, col] = prev_value
        else:
            prev_value = value

df2.head()

,2023-10,2023-09,2023-08,2023-07,2023-06,2023-05,2023-04,2023-03,2023-02,2023-01,...,1997-07,1997-06,1997-05,1997-04,1997-03,1997-02,1997-01,1996-12,1996-11,1996-10
0,10.0,10.0,10.0,10.0,10.0,10.0,10.0,1.0,1.0,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1.0,1.0,4.0,4.0,4.0,4.0,4.0,4.0,4.0,4.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,6.0,6.0,6.0,6.0,6.0,8.0,8.0,8.0,8.0,8.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,6.0,6.0,6.0,4.0,4.0,1.0,1.0,1.0,1.0,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,-1.0,-1.0,NaN,NaN
4,36.0,36.0,36.0,36.0,36.0,36.0,36.0,36.0,21.0,21.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [74]:
print(df2.sum())

2023-10    14712.0
2023-09    12506.0
2023-08    11827.0
2023-07    11209.0
2023-06    10466.0
            ...   
1997-02      -40.0
1997-01      -49.0
1996-12     -172.0
1996-11      -46.0
1996-10      -13.0
Length: 325, dtype: float64


In [ ]:
df2.to_csv(f"count_csvs/prelim_{keyword}_simsearch3.csv")